In [1]:
using Gmsh: gmsh
using Gridap
using GridapGmsh
using Gridap.Algebra
using Gridap.TensorValues
using Gridap.ReferenceFEs
using LinearAlgebra
using PyPlot

In [2]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
L = 1.0
h = 0.25
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, L, 0, h)
p4 = gmsh.model.geo.addPoint(0, L, 0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4])
ps1 = gmsh.model.geo.addPlaneSurface([cl1])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])
pg2 = gmsh.model.addPhysicalGroup(1, [l1])
pg3 = gmsh.model.addPhysicalGroup(1, [l3])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "support")
gmsh.model.setPhysicalName(1, pg3, "load")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("ExampleModel.msh")
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Done meshing 1D (0.000288 s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (0.00198 s)
Info    : 29 vertices 60 elements
Info    : Writing 'ExampleModel.msh'...
Info    : Done writing 'ExampleModel.msh'


In [3]:
model = GmshDiscreteModel("ExampleModel.msh")
writevtk(model,"Model1")

Info    : Reading 'ExampleModel.msh'...
Info    : 9 entities
Info    : 29 nodes
Info    : 48 elements
Info    : Done reading 'ExampleModel.msh'


3-element Vector{Vector{String}}:
 ["Model1_0.vtu"]
 ["Model1_1.vtu"]
 ["Model1_2.vtu"]

In [4]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)

40-element Vector{Int8}:
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [5]:
p = get_vertex_node(model)
gr = get_grid(model)
nodes = get_node_coordinates(gr);

In [6]:
order = 1

reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
V0 = TestFESpace(model,reffe;
  conformity=:H1,
  dirichlet_tags=["support","load"],
  dirichlet_masks=[(true,true), (true,false)])

g1(x) = VectorValue(0.0,0.0)
g2(x) = VectorValue(0.01,0.0)

U = TrialFESpace(V0,[g1,g2])

TrialFESpace()

In [7]:
const E = 70.0e9
const ν = 0.33
const λ = (E*ν)/((1+ν)*(1-2*ν))
const μ = E/(2*(1+ν))
σ(ε) = λ*tr(ε)*one(ε) + 2*μ*ε

σ (generic function with 1 method)

In [8]:
function Eigen(E)
    E2 = get_array.(E)
    Egn = eigvals.(E2)
    Egn1 = eigvecs.(E2)
    σ1 = map(x->x[1], Egn)
    σ2 = map(x->x[2], Egn)
    n1 = map(x->x[:,1], Egn1)
    n2 = map(x->x[:,2], Egn1)
    return σ1, σ1, n1, n2   
end

Eigen (generic function with 1 method)

In [9]:
function σEigen(ε)
    σElas = σ∘(ε)
    σEval = evaluate(σElas,nodes)
    σ1, σ1, n1, n2 = Eigen(σEval)
    σEg = σ1*(n1 ⊗ n1) + σ2*(n2 ⊗ n2) 
    return σEg
end

σEigen (generic function with 1 method)

In [10]:
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [11]:
a(u,v) = ∫( ε(v) ⊙ (σEigen∘(ε(u)) ))*dΩ
l(v) = 0

l (generic function with 1 method)

In [12]:
op = AffineFEOperator(a,l,U,V0)
uh = solve(op)

LoadError: 

It is not possible to perform the operation "σEigen" on the given cell fields.

See the caught error for more information. (If you are using the Visual
  Studio Code REPL you might not see the caught error, please use the
  command-line REPL instead).


In [13]:
writevtk(Ω,"results",cellfields=["uh"=>uh,"epsi"=>ε(uh),"sigma"=>σ∘ε(uh)])

LoadError: UndefVarError: uh not defined